In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0).strftime("%Y%m%d")
today

'20250519'

In [11]:
available_devices = ['Android Mobile', 'Apple Mobile', 'Android TV', 'Apple TV', 'Fire TV', 'Smart TV', 'Vizio TV', 'Roku', 'Web']

# Import Connect TV TCs and process accordingly
df_connectedTV_raw = pd.read_csv("data/KOCOWA_4.0_tc_connectedTV.csv").reset_index(drop=True)
df_connectedTV_raw = df_connectedTV_raw.rename(columns={"FireTV": "Fire TV", "ROKU": "Roku", "AndroidTV": "Android TV", "AppleTV": "Apple TV"})

# Import Web and Mobile TCs and process accordingly
df_webMobile_raw = pd.read_csv("data/KOCOWA_4.0_tc_webMobile_Edited.csv").reset_index(drop=True)
df_webMobile_raw = df_webMobile_raw.rename(columns={"WEB": "Web", "Apple_Mobile": "Apple Mobile", "Android_Phone": "Android Mobile"})
df_K4_webMobile = df_webMobile_raw.fillna(0).copy()

# Column adjustment: Connected TVs 
df_K4_connectedTV = df_connectedTV_raw.fillna(0).copy()
df_K4_connectedTV["Web"] = False
df_K4_connectedTV["Apple Mobile"] = False
df_K4_connectedTV["Android Mobile"] = False
df_K4_connectedTV["Smart TV"] = True
df_K4_connectedTV["Vizio TV"] = True
df_K4_connectedTV["project_name"] = "KOCOWA 4.0"

# Column adjustment: Web, Mobile
df_K4_webMobile["Android TV"] = False
df_K4_webMobile["Apple TV"] = False
df_K4_webMobile["Fire TV"] = False
df_K4_webMobile["Smart TV"] = False
df_K4_webMobile["Vizio TV"] = False
df_K4_webMobile["Roku"] = False
df_K4_webMobile["project_name"] = "KOCOWA 4.0"

df_K4_consolidated = pd.concat([df_K4_connectedTV, df_K4_webMobile], axis=0)

# Convert "UNTESTED" to True boolean flags
columns_to_check = df_K4_consolidated.columns[6:15]

# Replace "UNTESTED" with True and "비대상" with False in the specified columns
df_K4_consolidated[columns_to_check] = df_K4_consolidated[columns_to_check].replace({
    "UNTESTED": True,
    "비대상": False
})

df_K4_consolidated

/var/folders/lk/3z63ntmd7jx9rxfvb1tyzjb00000gn/T/ipykernel_24619/1050556093.py:36: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_K4_consolidated[columns_to_check] = df_K4_consolidated[columns_to_check].replace({


,대분류,중분류,소분류,Section,테스트 항목,기대결과,Fire TV,Roku,Android TV,Apple TV,Web,Apple Mobile,Android Mobile,Smart TV,Vizio TV,project_name
0,Splash Screen,0,0,Splash Screen,KOCOWA Logo image 확인,KOCOWA logo image 노출된다,True,True,True,True,False,False,False,True,True,KOCOWA 4.0
1,Splash Screen,0,0,Splash Screen,KOCOWA Logo image Animation 확인,KOCOWA logo image Animation 적용되어 노출된다,True,True,True,True,False,False,False,True,True,KOCOWA 4.0
2,Splash Screen,Get Started,0,Splash Screen>Get Started,Get Started Page 진입 동작 확인,Get Started Page 진입된다,True,True,True,True,False,False,False,True,True,KOCOWA 4.0
3,Splash Screen,Get Started,0,Splash Screen>Get Started,미결제 시 Initial Page 진입 동작 확인,Initial Page 진입된다,True,True,True,True,False,False,False,True,True,KOCOWA 4.0
4,Splash Screen,Get Started,0,Splash Screen>Get Started,Home Main Page 진입 동작 확인,Home Main Page 진입된다,True,True,True,True,False,False,False,True,True,KOCOWA 4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,My KOCOWA Page,Download Screen,Share POP UP,0,우측 Rolling 동작 확인,우측 Rolling 된다,False,False,False,False,False,True,True,False,False,KOCOWA 4.0
919,My KOCOWA Page,Download Screen,Share POP UP,0,우측 Rolling 동작 불가 여부 확인,우측 Rolling 동작하지 않는다,False,False,False,False,False,True,True,False,False,KOCOWA 4.0
920,My KOCOWA Page,Download Screen,Share POP UP,0,좌측 Rolling 동작 확인,좌측 Rolling 된다,False,False,False,False,False,True,True,False,False,KOCOWA 4.0
921,My KOCOWA Page,Download Screen,Empty Screen,0,title text 확인,"""Contents Download & Offline Playback"" text 출력된다",False,False,False,False,False,True,True,False,False,KOCOWA 4.0


## Import Connected TV TCs and add other device columns

### Group by main and sub categories to create `df_dev_component`

#### test cases will be joined by ", "

In [12]:
available_devices = ['Android Mobile', 'Apple Mobile', 'Android TV', 'Apple TV', 'Fire TV', 'Smart TV', 'Vizio TV', 'Roku', 'Web']

# Import Connect TV TCs and process accordingly
df_connectedTV_raw = pd.read_csv("data/KOCOWA_4.0_tc_connectedTV.csv").reset_index(drop=True)
df_connectedTV_raw = df_connectedTV_raw.rename(columns={"FireTV": "Fire TV", "ROKU": "Roku", "AndroidTV": "Android TV", "AppleTV": "Apple TV"})

# Import Web and Mobile TCs and process accordingly
df_webMobile_raw = pd.read_csv("data/KOCOWA_4.0_tc_webMobile_Edited.csv").reset_index(drop=True)
df_webMobile_raw = df_webMobile_raw.rename(columns={"WEB": "Web", "Apple_Mobile": "Apple Mobile", "Android_Phone": "Android Mobile"})
df_K4_webMobile = df_webMobile_raw.fillna(0).copy()

# Column adjustment: Connected TVs 
df_K4_connectedTV = df_connectedTV_raw.fillna(0).copy()
df_K4_connectedTV["Web"] = False
df_K4_connectedTV["Apple Mobile"] = False
df_K4_connectedTV["Android Mobile"] = False
df_K4_connectedTV["Smart TV"] = True
df_K4_connectedTV["Vizio TV"] = True
df_K4_connectedTV["project_name"] = "KOCOWA 4.0"

# Column adjustment: Web, Mobile
df_K4_webMobile["Android TV"] = False
df_K4_webMobile["Apple TV"] = False
df_K4_webMobile["Fire TV"] = False
df_K4_webMobile["Smart TV"] = False
df_K4_webMobile["Vizio TV"] = False
df_K4_webMobile["Roku"] = False
df_K4_webMobile["project_name"] = "KOCOWA 4.0"

df_K4_consolidated = pd.concat([df_K4_connectedTV, df_K4_webMobile], axis=0)

# Replace "UNTESTED" with True and "비대상" with False in the specified columns
df_K4_consolidated[columns_to_check] = df_K4_consolidated[columns_to_check].replace({
    "UNTESTED": True,
    "비대상": False
})

df_K4_consolidated = df_K4_consolidated.rename(columns={
    "대분류": "main_category",
    "중분류": "sub_category",
    "소분류": "component",
    "Section": "scenario",
    "테스트 항목": "test_case",
    "기대결과": "expected_result"})

device_columns = ['Fire TV', 'Roku', 'Android TV', 'Apple TV', 'Web','Apple Mobile', 'Android Mobile', 'Smart TV', 'Vizio TV']
col_orders = ['project_name','main_category', 'component', 'test_case', 'Fire TV', 'Roku', 'Android TV', 'Apple TV', 'Web','Apple Mobile', 'Android Mobile', 'Smart TV', 'Vizio TV']

# If component = 0, then fill it with sub_category
for index, row in df_K4_consolidated.iterrows():
    main_category = row.main_category
    component = row.component

    if main_category == "Contents":
        main_category_rename = "Content Details"
        df_K4_consolidated.at[index, "main_category"] = main_category_rename
    elif main_category == "Series Detailed Page":
        main_category_rename = "Content Details"
        df_K4_consolidated.at[index, "main_category"] = main_category_rename

    # Fill missing sub categories and components
    if component == 0:
        if row.sub_category == 0:
            component_rename = row.main_category
            df_K4_consolidated.at[index, "sub_category"] = component_rename
        else:
            component_rename = row.sub_category
        df_K4_consolidated.at[index, "component"] = component_rename

# Rule: If the name of the component is the same as main category, use "General Rule" tag
for index, row in df_K4_consolidated.iterrows():
    main_category = row.main_category
    component = row.component

    if component.strip().lower() == main_category.strip().lower():
        component_rename = "General Rule"
        df_K4_consolidated.at[index, "component"] = component_rename
    
    '''
    # Rule for 4.0 Project: if main category is Content Details and My KOCOWA, True for all web and mobile platforms
    if main_category == "My KOCOWA Page":
        df_K4_connectedTV.at[index, "Web"] = True
        df_K4_connectedTV.at[index, "Apple Mobile"] = True
        df_K4_connectedTV.at[index, "Android Mobile"] = True
    elif main_category == "Content Details":
        df_K4_connectedTV.at[index, "Web"] = True
        df_K4_connectedTV.at[index, "Apple Mobile"] = True
        df_K4_connectedTV.at[index, "Android Mobile"] = True
    '''

# Grouping the dataframe by main_category, sub_category, and component
# Aggregating test_case by joining them with ", " and taking the maximum value for device_columns
df_dev_component = (
    df_K4_consolidated
    .groupby(['project_name','main_category', 'component'])
    .agg({
        'test_case': lambda x: '\n'.join(x),  # Combine test cases into a single string
        **{col: 'max' for col in device_columns}  # Take the maximum value for each device column
    })
    .reset_index()  # Reset the index to make it a regular dataframe
).sort_values(by = ["main_category", "component"]).copy()
        
df_dev_component_cleaned = df_dev_component[df_dev_component.component != "General Rule"][col_orders].copy()

/var/folders/lk/3z63ntmd7jx9rxfvb1tyzjb00000gn/T/ipykernel_24619/1778389681.py:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_K4_consolidated[columns_to_check] = df_K4_consolidated[columns_to_check].replace({


In [13]:
len(list(df_dev_component.main_category.unique()))

14

In [16]:
df_dev_component_cleaned

,project_name,main_category,component,test_case,Fire TV,Roku,Android TV,Apple TV,Web,Apple Mobile,Android Mobile,Smart TV,Vizio TV
0,KOCOWA 4.0,Content Details,General Rule,최하단 Curation Rail에서 하단 키 동작 확인\n1st Curation R...,True,True,True,True,False,False,False,True,True
1,KOCOWA 4.0,Content Details,Accept Button,Accept Button 확인\nAccept Button stringset 확인\n...,True,True,True,True,False,False,False,True,True
2,KOCOWA 4.0,Content Details,Back Button,Back Button 확인\nBack Button stringset 확인\nBack...,True,True,True,True,False,False,False,True,True
3,KOCOWA 4.0,Content Details,Back icon,Trailer Player Back icon 버튼 노출 확인\nTrailer Pla...,False,False,False,False,False,True,True,False,False
4,KOCOWA 4.0,Content Details,Bookmarked icon,Bookmarked icon 확인\nBookmarked icon 선택 시 동작 확인...,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,KOCOWA 4.0,VARIETY Channel Page,Content List Area,Scroll Down 동작 확인\nScroll Up 동작 확인\nposter ima...,True,True,True,True,False,False,False,True,True
276,KOCOWA 4.0,VARIETY Channel Page,Sorting Button,Default Setting 확인\nSorting Button 확인\nSorting...,True,True,True,True,False,False,False,True,True
277,KOCOWA 4.0,VARIETY Channel Page,Sorting POP UP,Default Setting 확인\nCheck icon 확인\n리모컨 아래 방향키 ...,True,True,True,True,False,False,False,True,True
278,KOCOWA 4.0,VARIETY Channel Page,Top Area,Division Line 확인,True,True,True,True,False,False,False,True,True


### Initial export for checkup (csv and excel): processed_KOCOWA_4.0_tc_connectedTV

In [17]:
df_dev_component_cleaned.to_csv("data/processed/processed_KOCOWA_4.0_tc_connectedTV.csv", index=False)
# df_dev_component.to_excel("data/processed/processed_KOCOWA_4.0_tc_connectedTV.xlsx", index=False)